In [1]:
%cd ..

c:\Users\youse\Desktop\Research\Apperception\apperception


In [2]:
# Deleting the .apperception_cache if it exists, as to avoid DB conflict errors
import os
import shutil

sys.path.append(os.path.join(os.getcwd(),"apperception"))

dirpath = os.path.join('.apperception_cache')
if os.path.exists(dirpath) and os.path.isdir(dirpath):
    shutil.rmtree(dirpath)

dirpath = os.path.join('output')
if os.path.exists(dirpath) and os.path.isdir(dirpath):
    shutil.rmtree(dirpath)
os.mkdir(dirpath)

# This piece of code is unsafe, and should not be run if not needed. 
# It serves for test purposes when one recieves a "dead kernel" error.
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [3]:
##### NUSCENES DATA #####
from new_world import empty_world, World
from scenic_util import transformation
from camera import Camera
import pandas as pd
import pickle

World.db.reset()

name = 'ScenicWorld' # world name
units = 'metrics'      # world units
user_data_dir = os.path.join("v1.0-mini")

with open('df_sample_data.pickle', "rb") as f:
    df_sample_data = pickle.loads(f.read())
with open('df_annotation.pickle', "rb") as f:
    df_annotation = pickle.loads(f.read())


world = empty_world(name=name)

from camera_config import fetch_camera_config
# scenes = ["scene-0061", "scene-0103","scene-0553", "scene-0655", "scene-0757", "scene-0796", "scene-0916", "scene-1077", "scene-1094", "scene-1100"]
scenes = ["scene-0061"]
for scene in scenes:
    config = fetch_camera_config(scene, df_sample_data)
    camera = Camera(config=config, id=scene)
    world = world.add_camera(camera)
    df_config = df_sample_data[df_sample_data['scene_name'] == scene][['sample_token']]
    df_ann = df_annotation.join(df_config.set_index('sample_token'), on='sample_token', how='inner')
    world = world.recognize(camera, df_ann)


get backend Qt5Agg


In [9]:
df_annotation

,token,instance_token,translation,size,rotation,heading,category,camera_heading,token_sample_data
sample_token,,,,,,,,,
cd21dbfc3bd749c7b10a5c42562e0c42,70aecbe9b64f4722ab3c230391a3beb8,6dd2cbf4c24b4caeb625035869bca7b5,"[373.214, 1130.48, 1.25]","[0.621, 0.669, 1.642]","[0.9831098797903927, 0.0, 0.0, -0.183016295062...",338.909000,human.pedestrian.adult,359.631893,e0bc3ff06fe348fdb56039c640be0ac3
cd21dbfc3bd749c7b10a5c42562e0c42,70aecbe9b64f4722ab3c230391a3beb8,6dd2cbf4c24b4caeb625035869bca7b5,"[373.214, 1130.48, 1.25]","[0.621, 0.669, 1.642]","[0.9831098797903927, 0.0, 0.0, -0.183016295062...",338.909000,human.pedestrian.adult,359.631893,ebf7973f82f145df97b67c7af559b1a8
cd21dbfc3bd749c7b10a5c42562e0c42,70aecbe9b64f4722ab3c230391a3beb8,6dd2cbf4c24b4caeb625035869bca7b5,"[373.214, 1130.48, 1.25]","[0.621, 0.669, 1.642]","[0.9831098797903927, 0.0, 0.0, -0.183016295062...",338.909000,human.pedestrian.adult,359.631893,8b4d4aef4cdd4dc68340549326282c9a
cd21dbfc3bd749c7b10a5c42562e0c42,70aecbe9b64f4722ab3c230391a3beb8,6dd2cbf4c24b4caeb625035869bca7b5,"[373.214, 1130.48, 1.25]","[0.621, 0.669, 1.642]","[0.9831098797903927, 0.0, 0.0, -0.183016295062...",338.909000,human.pedestrian.adult,359.631893,4d41b098087948bda699b40ff62ed88a
cd21dbfc3bd749c7b10a5c42562e0c42,70aecbe9b64f4722ab3c230391a3beb8,6dd2cbf4c24b4caeb625035869bca7b5,"[373.214, 1130.48, 1.25]","[0.621, 0.669, 1.642]","[0.9831098797903927, 0.0, 0.0, -0.183016295062...",338.909000,human.pedestrian.adult,359.631893,ad15f0f3a94040d5844111b364eb02c4
...,...,...,...,...,...,...,...,...,...
dc4c80e8a1534ff8a3eb80307341161c,b8dd2792bd4a4b1eb4deba5083bf74f4,88b7a3d6e9394198a1511642a5aa5499,"[1324.369, 914.564, 1.106]","[1.836, 4.416, 1.43]","[0.8972826104991134, 0.0, 0.0, 0.4414565855165...",52.393708,vehicle.car,0.914443,487b649c38f549b1bb3cb5bb103e06d6
dc4c80e8a1534ff8a3eb80307341161c,b8dd2792bd4a4b1eb4deba5083bf74f4,88b7a3d6e9394198a1511642a5aa5499,"[1324.369, 914.564, 1.106]","[1.836, 4.416, 1.43]","[0.8972826104991134, 0.0, 0.0, 0.4414565855165...",52.393708,vehicle.car,0.914443,9a424dcc6eea4121b95dd7866b3a2747
dc4c80e8a1534ff8a3eb80307341161c,b8dd2792bd4a4b1eb4deba5083bf74f4,88b7a3d6e9394198a1511642a5aa5499,"[1324.369, 914.564, 1.106]","[1.836, 4.416, 1.43]","[0.8972826104991134, 0.0, 0.0, 0.4414565855165...",52.393708,vehicle.car,0.914443,258d3c52c8394ee588cbb8143c31c0e9


In [7]:
df_sample_data

,token,sample_token,timestamp,filename,is_key_frame,camera_translation,camera_rotation,camera_intrinsic,ego_rotation,ego_translation,scene_name,frame_order,heading
0,512015c209c1490f906982c3b182c2a8,39586f9d59004284a7114a68825e8eec,1532402927762460,sweeps/CAM_FRONT/n015-2018-07-24-11-22-45+0800...,False,"[1.70079118954, 0.0159456324149, 1.51095763913]","[0.4998015430569128, -0.5030316162024876, 0.49...","[[1266.417203046554, 0.0, 816.2670197447984], ...","[0.5731318069963492, -0.001008743951274553, 0....","[410.9433704785875, 1179.9098624203461, 0.0]",scene-0061,3,356.513909
1,4c48e46e76b74a4c895b906418f79e4b,356d81f38dd9473ba590f39e266f54e5,1532402928412460,sweeps/CAM_FRONT/n015-2018-07-24-11-22-45+0800...,False,"[1.70079118954, 0.0159456324149, 1.51095763913]","[0.4998015430569128, -0.5030316162024876, 0.49...","[[1266.417203046554, 0.0, 816.2670197447984], ...","[0.5756004222270319, 0.002468184877516363, 0.0...","[408.9502699908764, 1174.4933020919336, 0.0]",scene-0061,11,356.519911
2,31280aeef5f04d3cb00724081dcca0e4,e0845f5322254dafadbbed75aaa07969,1532402929112460,sweeps/CAM_FRONT/n015-2018-07-24-11-22-45+0800...,False,"[1.70079118954, 0.0159456324149, 1.51095763913]","[0.4998015430569128, -0.5030316162024876, 0.49...","[[1266.417203046554, 0.0, 816.2670197447984], ...","[0.5812651129089583, 0.0004956185578443312, 0....","[406.9547611089034, 1168.8764471325978, 0.0]",scene-0061,19,356.533823
3,31b823dfb6bc489e8c85272ef027deda,f1e3d9d08f044c439ce86a2d6fcca57b,1532402929762460,sweeps/CAM_FRONT/n015-2018-07-24-11-22-45+0800...,False,"[1.70079118954, 0.0159456324149, 1.51095763913]","[0.4998015430569128, -0.5030316162024876, 0.49...","[[1266.417203046554, 0.0, 816.2670197447984], ...","[0.5839935942635103, 0.0001221605241272317, 0....","[405.242155872287, 1163.862806857947, 0.0]",scene-0061,27,356.540547
4,1464f2f69fc54d469a9810ef77655098,4f545737bf3347fbbc9af60b0be9a963,1532402930412460,sweeps/CAM_FRONT/n015-2018-07-24-11-22-45+0800...,False,"[1.70079118954, 0.0159456324149, 1.51095763913]","[0.4998015430569128, -0.5030316162024876, 0.49...","[[1266.417203046554, 0.0, 816.2670197447984], ...","[0.5881335809828381, -0.0009546940046563173, 0...","[403.6676139166417, 1159.200310831268, 0.0]",scene-0061,35,356.550770
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14003,2f0c41630ed049c79d2c9713b62293ac,8336b62c3f6f4bb988c3865cc5df4446,1542801004604844,sweeps/CAM_FRONT_LEFT/n015-2018-11-21-19-38-26...,False,"[1.52387798135, 0.494631336551, 1.50932822144]","[0.6757265034669446, -0.6736266522251881, 0.21...","[[1272.5979470598488, 0.0, 826.6154927353808],...","[0.21734463639137175, 0.021947100873110408, 0....","[1320.4899619460525, 903.2850514824449, 0.0]",scene-1100,199,356.688527
14004,0d0438d077e3481db68b8c04de06cf83,8cc06390249d4895b5b9b860f0952f24,1542801005254844,sweeps/CAM_FRONT_LEFT/n015-2018-11-21-19-38-26...,False,"[1.52387798135, 0.494631336551, 1.50932822144]","[0.6757265034669446, -0.6736266522251881, 0.21...","[[1272.5979470598488, 0.0, 826.6154927353808],...","[0.21734590189901404, 0.02201001850017383, 0.0...","[1320.4899769167264, 903.2850689140539, 0.0]",scene-1100,207,356.688529
14005,0247ec5fa603476a9c9f5b8da9a422ad,d7cb9aa06de1442d8e2a22d562045cb4,1542801005904844,samples/CAM_FRONT_LEFT/n015-2018-11-21-19-38-2...,True,"[1.52387798135, 0.494631336551, 1.50932822144]","[0.6757265034669446, -0.6736266522251881, 0.21...","[[1272.5979470598488, 0.0, 826.6154927353808],...","[0.21736779146486868, 0.02205034298791243, 0.0...","[1320.4899595637255, 903.2850693428472, 0.0]",scene-1100,215,356.688576
14006,276a3634d746414a8e06cb7ca9e727a2,0d6668bcb0644bbaaee81c17cc308845,1542801006604844,sweeps/CAM_FRONT_LEFT/n015-2018-11-21-19-38-26...,False,"[1.52387798135, 0.494631336551, 1.50932822144]","[0.6757265034669446, -0.6736266522251881, 0.21...","[[1272.5979470598488, 0.0, 826.6154927353808],...","[0.21735700739688674, 0.022021890759692117, 0....","[1320.4898559087292, 903.2850283180987, 0.0]",scene-1100,223,356.68855

In [4]:
cams = world.get_camera()
# ids = world.get_id()
# print("cameras are", cams[:2])
# print("ids are", ids)

execute: insert_cam
New camera inserted successfully.........
execute: retrieve_cam
execute: get_cam
done execute node


In [5]:
import time
start_time = time.time()
### Scenic Code ###
# ego = Car
# Car offset by (Range(-10, 10), Range(20, 40))

### Apperception Query ###
filtered_world = world.filter_traj_type("vehicle.car")

# filtered_world = filtered_world.filter_camera(pred = lambda cam: filtered_world.roadDirection(cam.x, cam.y) == cam.heading)

filtered_world = filtered_world.filter_pred_relative_to_type(pred=lambda obj: (cam.x - 10) <= obj.x <= (cam.x + 10) 
                                                                          and (cam.y - 10) <= obj.y <= (cam.y + 0))

# filtered_world = filtered_world.filter_pred_relative_to_type(pred = lambda obj: -10 <= (obj.x - cam.x) <= 10
#                                                                             and 20 <= (obj.y - cam.y) <= 40
#                                                                             and obj.heading == cam.heading)

filtered_ids = filtered_world.get_traj_key()
print("filtered_ids are", filtered_ids)

print("----------------------------------------------------------------------")
print("Total execution time is: %s seconds" % (time.time() - start_time))
# print("Device Details: \n Processor: AMD Ryzen 7 5800H \n RAM Size: 16GB \n Graphics Card: NVIDIA GeForce RTX 3060 Laptop")
print("----------------------------------------------------------------------")

execute: insert_bbox_traj
Recognization done, saving to database......
execute: retrieve_traj
execute: filter_traj_type
execute: filter_relative_to_type
execute: get_traj_key
get_traj_key SELECT sq2.itemid FROM (SELECT DISTINCT sq1.* FROM (SELECT * FROM (SELECT * FROM Item_General_Trajectory WHERE cameraId='scene-0061') sq0 WHERE sq0.objecttype='vehicle.car') sq1 CROSS JOIN Cameras WHERE SQRT(POWER(ST_X(ST_Centroid(Cameras.egoTranslation))-valueAtTimestamp(getX(sq1.trajCentroids),CAST('2021-06-08T07:10:28' AS TIMESTAMPTZ)+Cameras.frameNum*CAST('1 second' AS INTERVAL)),2)+POWER(ST_Y(ST_Centroid(Cameras.egoTranslation))-valueAtTimestamp(getY(sq1.trajCentroids),CAST('2021-06-08T07:10:28' AS TIMESTAMPTZ)+Cameras.frameNum*CAST('1 second' AS INTERVAL)),2))*COS(PI()*Cameras.heading/180+ATAN((ST_Y(ST_Centroid(Cameras.egoTranslation))-valueAtTimestamp(getY(sq1.trajCentroids),CAST('2021-06-08T07:10:28' AS TIMESTAMPTZ)+Cameras.frameNum*CAST('1 second' AS INTERVAL)))/(ST_X(ST_Centroid(Cameras.egoT

In [11]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

traj = filtered_world.get_traj()
for i, id in enumerate(filtered_ids):
    print(id)
    filtered_world.overlay_trajectory(scenes[0], traj[i], id[0])

execute: insert_bbox_traj
execute: retrieve_traj
execute: filter_traj_type
execute: filter_relative_to_type
execute: get_traj
get_traj SELECT asMFJSON(trajCentroids)::json->'sequences' FROM (SELECT DISTINCT sq1.* FROM (SELECT * FROM (SELECT * FROM Item_General_Trajectory WHERE cameraId='scene-0061') sq0 WHERE sq0.objecttype='vehicle.car') sq1 CROSS JOIN Cameras WHERE SQRT(POWER(ST_X(ST_Centroid(Cameras.egoTranslation))-valueAtTimestamp(getX(sq1.trajCentroids),CAST('2021-06-08T07:10:28' AS TIMESTAMPTZ)+Cameras.frameNum*CAST('1 second' AS INTERVAL)),2)+POWER(ST_Y(ST_Centroid(Cameras.egoTranslation))-valueAtTimestamp(getY(sq1.trajCentroids),CAST('2021-06-08T07:10:28' AS TIMESTAMPTZ)+Cameras.frameNum*CAST('1 second' AS INTERVAL)),2))*COS(PI()*Cameras.heading/180+ATAN((ST_Y(ST_Centroid(Cameras.egoTranslation))-valueAtTimestamp(getY(sq1.trajCentroids),CAST('2021-06-08T07:10:28' AS TIMESTAMPTZ)+Cameras.frameNum*CAST('1 second' AS INTERVAL)))/(ST_X(ST_Centroid(Cameras.egoTranslation))-valueAtT

In [ ]:
### Scenic Code ###
# ego = Car
# Car offset by (Range(-10, 10), Range(20, 40)), 
# 	facing Range(-5, 5) deg

### Apperception Query ###
filtered_world = world.filter_traj_type("vehicle.car")

filtered_world = filtered_world.filter_camera(pred = lambda cam: filtered_world.roadDirection(cam.x, cam.y) == cam.heading)

filtered_world = filtered_world.filter_pred_relative_to_type(pred = lambda obj: -10 <= (obj.x - cam.x) <= 10
                                                                            and 20 <= (obj.y - cam.y) <= 40,
                                                             type="camera")
                                                             
filtered_world = filtered_world.filter_traj(pred = lambda obj: -5 <= obj.heading <= 5)

filtered_ids = filtered_world.get_traj_key()
print("filtered_ids are", filtered_ids)

In [ ]:
### Scenic Code ###
# ego = Car
# Car offset by (Range(-10, 10), Range(20, 40)), 
# 	facing Range(-5, 5) deg relative to ego

### Apperception Query ###
filtered_world = world.filter_traj_type("vehicle.car")

filtered_world = filtered_world.filter_camera(pred = lambda cam: filtered_world.roadDirection(cam.x, cam.y) == cam.heading)

filtered_world = filtered_world.filter_pred_relative_to_type(pred = lambda obj: -10 <= (obj.x - cam.x) <= 10
                                                                            and 20 <= (obj.y - cam.y) <= 40
                                                                            and -5 <= (obj.heading - cam.heading) <= 5,
                                                             type="camera")
                                                             
filtered_ids = filtered_world.get_traj_key()
print("filtered_ids are", filtered_ids)

In [ ]:
### Scenic Code ###
# ego = Car
# Car offset by (Range(-10, 10), Range(20, 40)), 
# 	facing Range(-5, 5) deg relative to roadDirection

### Apperception Query ###
filtered_world = world.filter_traj_type("vehicle.car")

filtered_world = filtered_world.filter_camera(pred = lambda cam: filtered_world.roadDirection(cam.x, cam.y) == cam.heading)

filtered_world = filtered_world.filter_pred_relative_to_type(pred = lambda obj: -10 <= (obj.x - cam.x) <= 10
                                                                            and 20 <= (obj.y - cam.y) <= 40,
                                                             type="camera")

filtered_world = filtered_world.filter_traj(pred = lambda obj: -5 <= (obj.heading - filtered_world.roadDirection(obj.x, obj.y)) <= 5)

filtered_ids = filtered_world.get_traj_key()
print("filtered_ids are", filtered_ids)

In [ ]:
### Scenic Code ###
# ego = car on road
# otherCar = Car ahead of ego by Range(4, 19)
# require not (otherCar in intersection)

### Apperception Query ###
filtered_world = world.filter_traj_type("vehicle.car")

filtered_world = filtered_world.filter_camera(pred = lambda cam: filtered_world.roadDirection(cam.x, cam.y) == cam.heading 
                                                             and filtered_world.roadType(cam.x, cam.y) != None)

filtered_world = filtered_world.filter_pred_relative_to_type(pred = lambda obj: 4 <= (obj.y - cam.y) <= 19,
                                                             type="camera")

filtered_world = filtered_world.filter_traj(pred = lambda obj: roadType(obj.x, obj.y) != RoadType.Intersection)

filtered_ids = filtered_world.get_traj_key()
print("filtered_ids are", filtered_ids)

In [ ]:
### Scenic Code ###
# ego = car in intersection
# otherCar = Car ahead of ego by Range(4, 19)
# require otherCar in rightLane

### Apperception Query ###
filtered_world = world.filter_traj_type("vehicle.car")

filtered_world = filtered_world.filter_camera(pred = lambda cam:  filtered_world.roadDirection(cam.x, cam.y) == cam.heading 
                                                              and filtered_world.roadType(cam.x, cam.y) == RoadType.Intersection)

filtered_world = filtered_world.filter_pred_relative_to_type(pred = lambda obj: 4 <= (obj.y - cam.y) <= 19,
                                                             type="camera")

filtered_world = filtered_world.filter_traj(pred = lambda obj: roadType(obj.x, obj.y) == RoadType.RightLane)

filtered_ids = filtered_world.get_traj_key()
print("filtered_ids are", filtered_ids)

In [ ]:
### Scenic Code ###
# def placeObjs(car, numCars):
#     for i in range(numCars):
#         car = Car ahead of car by Range(4, 5)
#         leftCar = Car left of car by Normal(2, 0.1), facing roadDirection
#         rightCar = Car right of car by Normal(3, 0.1), facing Range(0, 10) deg relative to ego.heading
#     return leftCar, rightCar

# spawn_point = 207.26 @ 8.72
# ego = Car at spawn_point, with visible_distance 200

# leftCar, rightCar = placeObjs(ego, 2)
# require (distance to leftCar) < 200
# require (distance to rightCar) < 200


In [ ]:
### Scenic Code ###
# def placeObjs(numPeds):
#     for i in range(numPeds):
#         Pedestrian offset by Range(-5, 5) @ Range(0, 200),
#             facing Range(-120, 120) deg relative to ego.heading

# spawn_point = 207.26 @ 8.72
# ego = Car at spawn_point,
#         with visibleDistance 200

# placeObjs(3)

In [ ]:
### Scenic Code ###
# ego = Car on drivableRoad,
#         facing Range(-15, 15) deg relative to roadDirection,
#         with visibleDistance 50, 
#         with viewAngle 135 deg
# ped = Pedestrian on roadsOrIntersections,
#         with regionContainedIn roadRegion,
#         facing Range(-180, 180) deg

# require abs(relative heading of ped from ego) > 70 deg

In [ ]:
### Scenic Code ###
# offset = Uniform(-1, 1) * Range(90, 180) deg

# ego = Car on drivableRoad,
#         facing offset relative to roadDirection,
#         with visibleDistance 50,
#         with viewAngle 135 deg

# otherCar = Car on visible road,
#             facing Range(-15, 15) deg relative to roadDirection

# require (distance from ego to otherCar) < 10

In [ ]:
### Scenic Code ###
# ego = Car on drivableRoad,
#         facing Range(-15, 15) deg relative to roadDirection,
#         with visibleDistance 50,
#         with viewAngle 135 deg

# other1 = Car on intersection,
#             facing -1 * Range(50, 135) deg relative to ego.heading

# other2 = Car on intersection,
#             facing -1 * Range(50, 135) deg relative to ego.heading

# require abs(relative heading of other1 from other2) > 100 deg
# require (distance from ego to intersectionRegion) < 10

In [ ]:
### Scenic Code ###
# ego = Car on drivableRoad,
#         facing Range(-15, 15) deg relative to roadDirection,
#         with visibleDistance 50,
#         with viewAngle 135 deg

# point1 = OrientedPoint ahead of ego by Range(0, 40)
# Car at (point1 offset by Range(-1, 1) & 0),
#     facing Range(-15, 15) deg relative to roadDirection

# oppositeCar = Car offset by (Range(-10, -1), Range(0, 50)),
#     facing Range(140, 180) deg relative to ego.heading

# point2 = OrientedPoint ahead of oppositeCar by Range(0, 40)
# Car at (point2 offset by Range(-1, 1) @ 0),
#     facing Range(-15, 15) deg relative to roadDirection

In [ ]:
### Scenic Code ###
# lanesWithRightLane = filter(lambda i: i._laneToRight, network.laneSections)
# egoLane = Uniform(*lanesWithRightLane)

# ego = Car on egoLane,
#         facing Range(-15, 15) deg relative to roadDirection
# cutInCar = Car offset by Range(0, 4) @ Range(0, 5),
#             facing -1*Range(15, 30) deg relative to roadDirection